# Weather Type

The purpose of this notebook is to plot the circulation patterns associated with the various weather types and the time series of weather types

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import calendar
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cartopy.crs as ccrs
import cartopy.feature
import paraguayfloodspy.visualize as viz # custom plotting library
import matplotlib.patches as patches
%matplotlib inline

Load in configuration parameters from file

In [ ]:
%run ../config/WeatherTypes.mk
print(WTEAST, WTWEST, WTSOUTH, WTNORTH)

Load in plotting options from file

In [ ]:
savefigs = True
%run ../config/PlotParameters.py
print(extent)
print(cmap)

Read in raw data

In [ ]:
psi850 = xr.open_dataset("../data/derived/psi_850.nc")
psi200 = xr.open_dataset("../data/derived/psi_200.nc")
prcp = xr.open_dataset('../data/derived/precip.nc')
wtypes = pd.read_csv("../data/derived/WeatherTypes.csv", index_col='time', parse_dates=True)

Get the X and Y coordinates for plotting

In [ ]:
X, Y = np.meshgrid(psi850.lon, psi850.lat)
Xp, Yp = np.meshgrid(prcp.lon, prcp.lat)

Print out the climatological probability of each weather type

In [ ]:
wt_counts = wtypes.groupby('wtype').size()
wt_counts2 = wtypes['2015-11-01':'2016-02-29'].groupby('wtype').size()
wt_prop = pd.DataFrame({'climatology': wt_counts / np.sum(wt_counts), 
              'NDJF1516': wt_counts2 / np.sum(wt_counts2)})

In [ ]:
plt.figure(figsize=(8.5, 4.5))
plt.plot(wt_prop['climatology'], label = "Climatology")
plt.plot(wt_prop['NDJF1516'], label = "2015-16")
plt.xlabel("Weather Type")
plt.ylabel("Proportion of Total Days")
plt.legend()
plt.grid()
if savefigs:
    plt.savefig("../figs/WT_Occurrence_Proportion.pdf")

In [ ]:
wt_prop.to_latex("../writeup/WeatherTypeProp.tex")
wt_prop

Repeat for NDJF 2015-16

## Composites

In [ ]:
wt_all = np.unique(wtypes['wtype'])
ncols = len(wt_all)

Give us a box to indicate the weather typing region

In [ ]:
def wt_region():
    rec = patches.Rectangle(
        (WTWEST-360, WTSOUTH),
        WTEAST - WTWEST,
        WTNORTH - WTSOUTH,
        color = 'blue', label = 'Weather Typing Region',
        fill = None, linewidth=2,
        transform = ccrs.PlateCarree()
    )
    return(rec)

Create the plot

In [ ]:
fig, axes = viz.SetupAxes(ncol = ncols, nax = 3 * ncols, proj = ccrs.Orthographic(-60, -10), figsize = [18, 7])
for i,wt in enumerate(wt_all):
    def selector(ds):
        times = wtypes.loc[wtypes['wtype'] == wt].index
        ds = ds.sel(time = np.in1d(ds.time, times))
        ds = ds.mean(dim = 'time')
        return(ds)

    # Row 1: 850 hPa wind
    ax = axes[0, i]
    ax.set_title('WT {}: {:.1%} of days'.format(wt, wt_prop['climatology'].values[i]))
    C0 = ax.contourf(X, Y, selector(psi200['anomaly']), transform = ccrs.PlateCarree(), 
                     cmap = cmap['psi_a'], extend="both", 
                     levels=np.linspace(-9e6, 9e6, 10))
    #
    ax = axes[1, i]
    C1 = ax.contourf(X, Y, selector(psi850['anomaly']), transform = ccrs.PlateCarree(), 
                     cmap = cmap['psi_a'], extend="both", 
                     levels=np.linspace(-4.5e6, 4.5e6, 10))
    #
    ax = axes[2, i]
    sub = selector(prcp['anomaly']).values
    sub = np.ma.masked_invalid(sub)
    C2 = ax.contourf(Xp, Yp, sub, transform = ccrs.PlateCarree(), 
                     cmap = cmap['rain_a'], extend='both', 
                     levels=np.linspace(-6, 6, 13))
plt.tight_layout()
fig.subplots_adjust(right=0.94)
cax0 = fig.add_axes([0.96, 0.7, 0.007, 0.25])
cax1 = fig.add_axes([0.96, 0.375, 0.007, 0.25])
cax2 = fig.add_axes([0.96, 0.05, 0.007, 0.25])
cbar0 = fig.colorbar(C0, cax = cax0)
cbar0.formatter.set_powerlimits((7, 7))
cbar0.update_ticks() 
cbar0.set_label(r'$\psi_{200}$ Anomaly [$m^2$/s]', rotation=270)
cbar0.ax.get_yaxis().labelpad = 20
#
cbar1 = fig.colorbar(C1, cax = cax1)
cbar1.formatter.set_powerlimits((6, 6))
cbar1.update_ticks() 
cbar1.set_label(r'$\psi_{850}$ Anomaly [$m^2$/s]', rotation=270)
cbar1.ax.get_yaxis().labelpad = 20
# 
cbar2 = fig.colorbar(C2, cax=cax2)
cbar2.set_label('Precip. Anomaly [mm/d]', rotation=270)
cbar2.ax.get_yaxis().labelpad = 20
# 
viz.FormatAxes(axes, extent = extent['SAm'])
#
if savefigs:
    fig.savefig("../figs/WTComposite.pdf", bbox_inches='tight')

## Time Series

In [ ]:
prcp_rpy = pd.read_csv("../data/derived/rainfall_rpy.csv", index_col='time')[['raw']]
prcp_rpy_sub = prcp_rpy['2015-11-01':'2016-02-29']

In [ ]:
wtypes_sub = wtypes['2015-11-01':'2016-02-29']
wt_prcp = prcp_rpy_sub.join(wtypes_sub)
wt_prcp.head()

In [ ]:
fig = plt.figure(figsize=(14,4))
plt.plot(wt_prcp.index, wt_prcp['raw'], color='gray', linewidth=1)
plt.axhline(np.nanpercentile(prcp_rpy['raw'], 50), label="p50", color='blue', linestyle='--', linewidth=0.75)
plt.axhline(np.nanpercentile(prcp_rpy['raw'], 90), label="p90", color='blue', linestyle='--', linewidth=0.75)
plt.axhline(np.nanpercentile(prcp_rpy['raw'], 99), label="p99", color='blue', linestyle='--', linewidth=0.75)
plt.scatter(wt_prcp.index, wt_prcp['raw'], c=wt_prcp['wtype'], cmap=plt.get_cmap('Accent', 6), s=65)
plt.ylabel('Area-Averaged Rainfall [mm/d]')
plt.grid(True)
for i,t in enumerate(wt_prcp.index):
    plt.text(t, wt_prcp.iloc[i]['raw'], '{:d}'.format(np.int(wt_prcp.iloc[i]['wtype'])))
if savefigs:
    fig.savefig("../figs/RainfallWeatherType.pdf", bbox_inches='tight')